<a href="https://colab.research.google.com/github/MixanBac/HPC2022/blob/main/Matmul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [4]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-gx_uaru8
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-gx_uaru8
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4307 sha256=b1c0b528f28a5cdd2afb94f1e36ef33ff3db65c27b17981f8afaec0d5cc0dc98
  Stored in directory: /tmp/pip-ephem-wheel-cache-efjd3jxd/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [6]:


!   nvidia-smi 
 



Thu Nov 17 15:09:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Как мы видим, Google Colab использует видеокарту Nvidia Tesla T4

In [7]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <cmath>
#include <time.h> 
#define BLOCK_SIZE  16          // submatrix size

__global__ void matMultCuda(float* a, float* b, int n, float* c)
{
    int bx = blockIdx.x;        // block index
    int by = blockIdx.y;

    int tx = threadIdx.x;       // thread index
    int ty = threadIdx.y;

    // Index of the first sub-matrix of A processed by the block
    int aBegin = n * BLOCK_SIZE * by;
    int aEnd = aBegin + n - 1;
    // Step size used to iterate through the sub-matrices of A
    int aStep = BLOCK_SIZE;
    // Index of the first sub-matrix of B processed by the block
    int bBegin = BLOCK_SIZE * bx;
    // Step size used to iterate through the sub-matrices of B
    int bStep = BLOCK_SIZE * n;
    float sum = 0.0f;           // computed subelement

    for (int ia = aBegin, ib = bBegin; ia <= aEnd; ia += aStep, ib += bStep)
    {
        // Shared memory for the sub-matrix of A
        __shared__ float as[BLOCK_SIZE][BLOCK_SIZE];
        // Shared memory for the sub-matrix of B
        __shared__ float bs[BLOCK_SIZE][BLOCK_SIZE];

        // Load the matrices from global memory to shared memory;
        as[ty][tx] = a[ia + n * ty + tx];
        bs[ty][tx] = b[ib + n * ty + tx];

        __syncthreads();    // Synchronize to make sure the matrices are loaded

                            // Multiply the two matrices together;
        for (int k = 0; k < BLOCK_SIZE; k++)
            sum += as[ty][k] * bs[k][tx];

        // Synchronize to make sure that the preceding
        // computation is done before loading two new
        // sub-matrices of A and B in the next iteration
        __syncthreads();
    }

    // Write the block sub-matrix to global memory;
    // each thread writes one element
    int ic = n * BLOCK_SIZE * by + BLOCK_SIZE * bx;

    c[ic + n * ty + tx] = sum;
}

double calcCuda(float* a, float* b, float* c, int N, bool flag)
{
    clock_t start2 = clock();
    int numBytes = N * N * sizeof(float);
    // allocate device memory
    float* adev = NULL;
    float* bdev = NULL;
    float* cdev = NULL;

    cudaMalloc((void**)&adev, numBytes);
    cudaMalloc((void**)&bdev, numBytes);
    cudaMalloc((void**)&cdev, numBytes);

    // set kernel launch configuration
    dim3 threads(BLOCK_SIZE, BLOCK_SIZE);
    dim3 blocks(N / threads.x, N / threads.y);

    // create cuda event handles
    cudaEvent_t start, stop;
    float gpuTime = 0.0f;

    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // asynchronously issue work to the GPU (all to stream 0)
    cudaEventRecord(start, 0);
    cudaMemcpy(adev, a, numBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(bdev, b, numBytes, cudaMemcpyHostToDevice);

    matMultCuda << < blocks, threads >> > (adev, bdev, N, cdev);

    cudaMemcpy(c, cdev, numBytes, cudaMemcpyDeviceToHost);
    cudaEventRecord(stop, 0);

    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&gpuTime, start, stop);

    // print the events gpu times
    if(flag)
    printf("Time spent executing by the GPU events: %.2f millseconds\n", gpuTime);

    // release resources
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(adev);
    cudaFree(bdev);
    cudaFree(cdev);
    clock_t end2 = clock();
    double millseconds2 = (double)(end2 - start2);
    // print the gpu times
    if (flag)
    printf("Time spent executing by the GPU: %.2f milliseconds\n", millseconds2);
    return millseconds2;
}

double calcCPU(float* a, float* b, float* c, int N)
{
    clock_t start3 = clock();
    for (int i = 0; i < N; ++i)
    {
        for (int j = 0; j < N; ++j)
        {
            c[i * N + j] = 0;
            for (int k = 0; k < N; ++k)
                c[i * N + j] += a[i * N + k] * b[k * N + j];
        }
    }
    clock_t end3 = clock();
    double millseconds = (double)(end3 - start3);
    // print the cpu times
    printf("Time spent executing by the CPU: %.2f milliseconds\n", millseconds);
    return millseconds;
}

int main(int argc, char* argv[])
{
    float* a = new float[64 * 64];
    float* b = new float[64 * 64];
    float* cpu = new float[64 * 64];
    float* gpu = new float[64 * 64];
    //Run to initialize cuda
    calcCuda(a, b, gpu, 64, false);
    //Main
    for (int i = 6; i < 12; i++) 
    {
        int N = pow(2, i);       // matrix size is N*N
        printf("Experiment for matrix size: %u \n", N);
        // allocate host memory
        float* a = new float[N * N];
        float* b = new float[N * N];
        float* cpu = new float[N * N];
        float* gpu = new float[N * N];

        for (int i = 0; i < N; i++)
            for (int j = 0; j < N; j++)
            {
                a[i * N + j] = rand()%100;
                b[i * N + j] = rand()%100;
            }
        printf("Acceleration factor: %.2f \n", calcCPU(a, b, cpu, N)/calcCuda(a, b, gpu, N, true));
        bool rel = true;
        for (int i = 0; i < N; i++)
            for (int j = 0; j < N; j++)
            {
                if (cpu[i * N + j] != gpu[i * N + j]) { rel = false; break; }
            }
        printf("Relevance: %s \n", rel ? "true" : "false");
        delete a;
        delete b;
        delete cpu;
        delete gpu;
    }

    return 0;
}

Experiment for matrix size: 64 
Time spent executing by the CPU: 1200.00 milliseconds
Time spent executing by the GPU events: 0.06 millseconds
Time spent executing by the GPU: 269.00 milliseconds
Acceleration factor: 4.46 
Relevance: true 
Experiment for matrix size: 128 
Time spent executing by the CPU: 10102.00 milliseconds
Time spent executing by the GPU events: 0.12 millseconds
Time spent executing by the GPU: 333.00 milliseconds
Acceleration factor: 30.34 
Relevance: true 
Experiment for matrix size: 256 
Time spent executing by the CPU: 83532.00 milliseconds
Time spent executing by the GPU events: 0.39 millseconds
Time spent executing by the GPU: 664.00 milliseconds
Acceleration factor: 125.80 
Relevance: true 
Experiment for matrix size: 512 
Time spent executing by the CPU: 885820.00 milliseconds
Time spent executing by the GPU events: 1.80 millseconds
Time spent executing by the GPU: 2415.00 milliseconds
Acceleration factor: 366.80 
Relevance: true 
Experiment for matrix size:

In [ ]:
params = {
    'A':            [64, 128, 256, 512, 1024, 2048],
    'acceleration factor':[4.26, 30.14, 119.57, 352.56, 946.96, 1998.34],
    'cpu_time':     [1154.00, 9917.00, 82266.00, 828156.00, 11066140.00, 134362099.00],
    'gpu_time':     [271.00, 329.00, 688.00, 2349.00, 11686.00, 67237.00],
}

exp_count = len(params["A"])

In [ ]:
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
data = []
for k in ['cpu_time', 'gpu_time']:
      plot = go.Scatter(x=params['A'],
                          y=params[k],
                          mode="lines+markers",
                          name=k
                          )
      data.append(plot)
layout = go.Layout(title="Dependence of time on the dimension of the matrix",
                   xaxis= dict(title= 'Matrix dimension',ticklen= 20,zeroline= False), 
                   yaxis= dict(title= 'Time, milliseconds',ticklen= 20,zeroline= False))
fig1 = go.Figure(data=data, layout = layout)
fig1.show()

data = []
plot = go.Scatter(x=params['A'],
                      y=params['acceleration factor'],
                      mode="lines+markers"
                      )
layout = go.Layout(title="Graph of acceleration from the dimension of the matrix",
                   xaxis= dict(title= 'Matrix dimension',ticklen= 20,zeroline= False), 
                   yaxis= dict(title= 'Acceleration factor',ticklen= 20,zeroline= False))
data.append(plot)
fig2 = go.Figure(data=data, layout = layout)
fig2.show()